## data loading, basic feature engineering

### Data loading & concat

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import os
import sys
import time
from typing import List, Optional

import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

from sklearn.model_selection import GroupKFold
from contextlib import contextmanager
from enum import Enum
from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm
from numba import jit


import pickle

@contextmanager
def timer(name: str):
    s = time.time()
    yield
    elapsed = time.time() - s
    print(f'[{name}] {elapsed: .3f}sec')

# pip install pyarrow

In [3]:
DATA_DIR = "C:/Users/po020/Desktop/KAIST/Python Codes/03. 학과공부/004. 빅데이터분석/02. 팀프로젝트/optiver-realized-volatility-prediction-1/"

In [4]:
# train data 수집

train = pd.read_csv(os.path.join(DATA_DIR, 'optiver_data', 'train.csv'))
stock_ids = set(train['stock_id'])

In [5]:
# data load 함수
    
def load_stock_data(stock_id, directory):
    return pd.read_parquet(os.path.join(DATA_DIR, 'optiver_data', directory, f'stock_id={stock_id}'))
    
def load_data(stock_id, stem, block):
    if block == 'train':
        return load_stock_data(stock_id, f'{stem}_train.parquet')
    elif block == 'test':
        return load_stock_data(stock_id, f'{stem}_test.parquet')
    else:
        return pd.concat([
            load_data(stock_id, stem, 'train'),
            load_data(stock_id, stem, 'test')
        ]).reset_index(drop=True)

def load_book(stock_id, block='train'):
    return load_data(stock_id, 'book', block)

def load_trade(stock_id, block='train'):
    return load_data(stock_id, 'trade', block)

In [188]:
temp = pd.DataFrame({'A': [1, 1, 2, 2], 'B': [1, 2, 3, 4], 'C': np.random.randn(4)})

In [189]:
temp.groupby('A').agg({'B':[np.sum, np.std], 'C': np.sum}).reset_index(drop=False)
temp.columns = flatten_name('book', temp.columns)

A   B                   C
     sum       std       sum
0  1   3  0.707107 -0.097271
1  2   7  0.707107 -0.740038

In [142]:
temp

,A,B,C
0,1,1,1.188552
1,1,2,-0.130229
2,2,3,-0.335772
3,2,4,0.843275


## Feature Engineering

### Base Features

In [14]:
# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# Function to calculate the log of the return
# Remember that logb(x / y) = logb(x) - logb(y)
def log_return(series: np.ndarray):
    return np.log(series).diff()

def log_return_df2(series: np.ndarray):
    return np.log(series).diff(2)

# prefix: book or trade
# src_names : feature 이름(columns 삽입 - agg.columns)
def flatten_name(prefix, src_names):
    ret = []
    for c in src_names:
        if c[0] in ['time_id', 'stock_id']:
            ret.append(c[0])
        else:
            ret.append('.'.join([prefix] + list(c))) # column 이름.book_
    return ret


def make_book_feature(stock_id, block = 'train'):
    book = load_book(stock_id, block)

    book['wap1'] = calc_wap1(book)
    book['wap2'] = calc_wap2(book)
    book['log_return1'] = book.groupby(['time_id'])['wap1'].apply(log_return)
    book['log_return2'] = book.groupby(['time_id'])['wap2'].apply(log_return)
    book['log_return_ask1'] = book.groupby(['time_id'])['ask_price1'].apply(log_return)
    book['log_return_ask2'] = book.groupby(['time_id'])['ask_price2'].apply(log_return)
    book['log_return_bid1'] = book.groupby(['time_id'])['bid_price1'].apply(log_return)
    book['log_return_bid2'] = book.groupby(['time_id'])['bid_price2'].apply(log_return)

    # Calculate wap balance
    book['wap_balance'] = abs(book['wap1'] - book['wap2'])
    # Calculate spread
    book['price_spread'] = (book['ask_price1'] - book['bid_price1']) / ((book['ask_price1'] + book['bid_price1']) / 2)
    book['bid_spread'] = book['bid_price1'] - book['bid_price2']
    book['ask_spread'] = book['ask_price1'] - book['ask_price2']
    book['total_volume'] = (book['ask_size1'] + book['ask_size2']) + (book['bid_size1'] + book['bid_size2'])
    book['volume_imbalance'] = abs((book['ask_size1'] + book['ask_size2']) - (book['bid_size1'] + book['bid_size2']))
    
    features = {
        'seconds_in_bucket': ['count'],
        'wap1': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'log_return_ask1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return_ask2': [np.sum, realized_volatility, np.mean, np.std],
        'log_return_bid1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return_bid2': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance': [np.sum, np.mean, np.std],
        'price_spread':[np.sum, np.mean, np.std],
        'bid_spread':[np.sum, np.mean, np.std],
        'ask_spread':[np.sum, np.mean, np.std],
        'total_volume':[np.sum, np.mean, np.std],
        'volume_imbalance':[np.sum, np.mean, np.std]
    }
    
    agg = book.groupby('time_id').agg(features).reset_index(drop=False)
    agg.columns = flatten_name('book', agg.columns)
    agg['stock_id'] = stock_id

# time별로 묶어서 feature 더 생성
    
    for time in [450, 300, 150]:
        d = book[book['seconds_in_bucket'] >= time].groupby('time_id').agg(features).reset_index(drop=False)
        d.columns = flatten_name(f'book_{time}', d.columns)
        agg = pd.merge(agg, d, on='time_id', how='left')
    return agg


def make_trade_feature(stock_id, block = 'train'):
    trade = load_trade(stock_id, block)
    trade['log_return'] = trade.groupby('time_id')['price'].apply(log_return)

    # Dict for aggregations
    features = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':['count'],
        'size':[np.sum],
        'order_count':[np.mean],
    }

    agg = trade.groupby('time_id').agg(features).reset_index()
    agg.columns = flatten_name('trade', agg.columns)
    agg['stock_id'] = stock_id
        
    for time in [450, 300, 150]:
        d = trade[trade['seconds_in_bucket'] >= time].groupby('time_id').agg(features).reset_index(drop=False)
        d.columns = flatten_name(f'trade_{time}', d.columns)
        agg = pd.merge(agg, d, on='time_id', how='left')
    return agg


# v2는 price가 들어감
# tick size를 계산해서 나오는 열 추가해줌.

def make_book_feature_v2(stock_id, block = 'train'):
    book = load_book(stock_id, block)

    prices = book.set_index('time_id')[['bid_price1', 'ask_price1', 'bid_price2', 'ask_price2']]
    time_ids = list(set(prices.index))

    ticks = {}
    for tid in time_ids:
        price_list = prices.loc[tid].values.flatten()
        price_diff = sorted(np.diff(sorted(set(price_list))))
        ticks[tid] = price_diff[0]
        
    dst = pd.DataFrame()
    dst['time_id'] = np.unique(book['time_id'])
    dst['stock_id'] = stock_id
    dst['tick_size'] = dst['time_id'].map(ticks)

    #book['tick_size'] = book['time_id'].map(ticks)
    
    # https://www.kaggle.com/lucasmorin/volatility-maximum-likelihood-estimation/comments#1495387
    #book['log_wap'] = np.log(calc_wap1(book))
    #book['log_return'] = book.groupby('time_id')['log_wap'].diff()
    #book['dt'] = book.groupby('time_id')['seconds_in_bucket'].diff()
    #book['inv_dt'] = 1 / book['dt']
    #book['estimated_volatility'] = np.power(book['log_return'] / np.sqrt(book['dt']), 2)
    #agg = book.groupby('time_id')['estimated_volatility'].sum()
    #dst['book.estimated_volatility'] = dst['time_id'].map(agg)
    #dst['book.estimated_volatility'] = np.sqrt(dst['book.estimated_volatility'])

    #book['log_return_df2'] = book.groupby(['time_id'])['wap1'].apply(log_return_df2)
    #book['price_spread_tick'] = (book['ask_price1'] - book['bid_price1']) / book['tick_size']
    #agg = book.groupby('time_id')['inv_dt'].mean()
    #dst['book.inv_dt.mean'] = dst['time_id'].map(agg)

    return dst

# trade 데이터의 size랑 order_count
# 근데 최종적으로 이사람은 추가 안함.

def make_trade_feature_v2(stock_id, block = 'train'):
    trade = load_trade(stock_id, block)
    trade['log_return'] = trade.groupby('time_id')['price'].apply(log_return)

    # Dict for aggregations
    features = {
        'size':[np.mean],
        'order_count':[np.sum],
    }

    agg = trade.groupby('time_id').agg(features).reset_index()
    agg.columns = flatten_name('trade', agg.columns)
    agg['stock_id'] = stock_id
        
    for time in [450, 300, 150]:
        d = trade[trade['seconds_in_bucket'] >= time].groupby('time_id').agg(features).reset_index(drop=False)
        d.columns = flatten_name(f'trade_{time}', d.columns)
        agg = pd.merge(agg, d, on='time_id', how='left')
    return agg

# 일단은 make_features 만 하면 괜춘할듯

def make_features(base, block):
    stock_ids = set(base['stock_id'])
    with timer('books'):
        books = Parallel(n_jobs=-1)(delayed(make_book_feature)(i, block) for i in stock_ids)
        book = pd.concat(books)

    with timer('trades'):
        trades = Parallel(n_jobs=-1)(delayed(make_trade_feature)(i, block) for i in stock_ids)
        trade = pd.concat(trades)

    with timer('extra features'):
        df = pd.merge(base, book, on=['stock_id', 'time_id'], how='left')
        df = pd.merge(df, trade, on=['stock_id', 'time_id'], how='left')
        #df = make_extra_features(df)
        
    return df

def make_features_v2(base, block):
    stock_ids = set(base['stock_id'])
    with timer('books(v2)'):
        books = Parallel(n_jobs=-1)(delayed(make_book_feature_v2)(i, block) for i in stock_ids)
        book_v2 = pd.concat(books)
    #with timer('trades(v2)'):
    #    trades = Parallel(n_jobs=-1)(delayed(make_trade_feature_v2)(i, block) for i in stock_ids)
    #    trade_v2 = pd.concat(trades)

    d = pd.merge(base, book_v2, on=['stock_id', 'time_id'], how='left')
    return d
    #return pd.merge(d, trade_v2, on=['stock_id', 'time_id'], how='left')

In [7]:
df = make_features(train, 'train')
df = make_features_v2(df, 'train')

[books]  1106.022sec
[trades]  153.998sec
[extra features]  2.849sec
[books(v2)]  80.117sec


In [8]:
# test set에 대해서 만들어줌

test = pd.read_csv(os.path.join(DATA_DIR, 'optiver_data', 'test.csv'))
# if len(test) == 3:
#     IS_1ST_STAGE = True

test_df = make_features(test, 'test')
test_df = make_features_v2(test_df, 'test')

print(df.shape)
print(test_df.shape)
df = pd.concat([df, test_df.drop('row_id', axis=1)]).reset_index(drop=True)

df['trade.tau'] = np.sqrt(1 / df['trade.seconds_in_bucket.count'])
df['trade_150.tau'] = np.sqrt(1 / df['trade_150.seconds_in_bucket.count'])
df['book.tau'] = np.sqrt(1 / df['book.seconds_in_bucket.count'])
# #df['book.log_return1.realized_volatility.sq'] = np.power(df['book.log_return1.realized_volatility'], 2)

[books]  0.206sec
[trades]  0.071sec
[extra features]  0.013sec
[books(v2)]  0.029sec
(428932, 216)
(3, 216)


### NN Features

In [111]:
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])

In [115]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)

In [116]:
distances

array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.41421356],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.41421356]])

In [117]:
indices

array([[0, 1],
       [1, 0],
       [2, 1],
       [3, 4],
       [4, 3],
       [5, 4]], dtype=int64)

In [9]:
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import minmax_scale

N_NEIGHBORS_MAX = 80

class Neighbors:
    def __init__(self, pivot, p, metric='minkowski', metric_params=None):
        nn = NearestNeighbors(n_neighbors=N_NEIGHBORS_MAX, p=p, metric=metric, metric_params=metric_params)
        nn.fit(pivot)
        self.distances, self.neighbors = nn.kneighbors(pivot, return_distance=True)

with timer('knn fit'):
    df_pv = df[['stock_id', 'time_id']].copy()
    df_pv['price'] = 0.01 / df['tick_size']
    df_pv['vol'] = df['book.log_return1.realized_volatility']
    df_pv['trade.tau'] = df['trade.tau']
    df_pv['trade.size.sum'] = df['book.total_volume.sum']
####################################################################################
# price neighbor
    pivot = df_pv.pivot('time_id', 'stock_id', 'price') # index: time_id, column: stock_id, value: price
    pivot = pivot.fillna(pivot.mean())
    pivot = pd.DataFrame(minmax_scale(pivot))
    
    k_neighbors_p2 = Neighbors(pivot, 2, metric='canberra')
    k_neighbors_p1 = Neighbors(pivot, 2, metric='mahalanobis', metric_params={'VI':np.cov(pivot.values.T)})
    k_neighbors_stock = Neighbors(minmax_scale(pivot.transpose()), 1)
####################################################################################
# vol neighbor
    pivot = df_pv.pivot('time_id', 'stock_id', 'vol')

    #pivot = pd.concat([df_pv.pivot('time_id', 'stock_id', 'vol'), df_pv.pivot('time_id', 'stock_id', 'trade.tau')], axis=1).copy()
    pivot = pivot.fillna(pivot.mean())
    pivot = pd.DataFrame(minmax_scale(pivot))
    
    k_neighbors_vol = Neighbors(pivot, 1)
    k_neighbors_stock_vol = Neighbors(minmax_scale(pivot.transpose()), 1)
####################################################################################
# trade.size.num neighbor
    pivot = df_pv.pivot('time_id', 'stock_id', 'trade.size.sum')
    pivot = pivot.fillna(pivot.mean())
    pivot = pd.DataFrame(minmax_scale(pivot))
    k_neighbors_size = Neighbors(pivot, 2, metric='mahalanobis', metric_params={'VI':np.cov(pivot.values.T)})
    k_neighbors_size_p2 = Neighbors(pivot, 2, metric='canberra')
    k_neighbors_stock_size = Neighbors(minmax_scale(pivot.transpose()), 1)


[knn fit]  169.697sec


In [15]:
def make_neighbors(df, k_neighbors, feature_col, n=5):
    feature_pivot = df.pivot('time_id', 'stock_id', feature_col)
    feature_pivot = feature_pivot.fillna(feature_pivot.mean())
    feature_pivot.head()
    
    neighbors = np.zeros((n, *feature_pivot.shape)) #dynamic instance : * -> 계속해서 변동할때 사용.

    for i in range(n):
        neighbors[i, :, :] += feature_pivot.values[k_neighbors[:, i], :]
        
    return feature_pivot, neighbors

def make_neighbors_stock(df, k_neighbors, feature_col, n=5):
    feature_pivot = df.pivot('time_id', 'stock_id', feature_col)
    feature_pivot = feature_pivot.fillna(feature_pivot.mean())
    feature_pivot.head()
    
    neighbors = np.zeros((n, *feature_pivot.shape))

    for i in range(n):
        neighbors[i, :, :] += feature_pivot.values[:, k_neighbors[:, i]]
        
    return feature_pivot, neighbors

def make_nn_feature(df, neighbors, columns, index, n=5, agg=np.mean, postfix='', exclude_self=False, exact=False):
    start = 1 if exclude_self else 0
    
    if exact:
        pivot_aggs = pd.DataFrame(neighbors[n-1,:,:], columns=columns, index=index)
    else:
        pivot_aggs = pd.DataFrame(agg(neighbors[start:n,:,:], axis=0), columns=columns, index=index)
    dst = pivot_aggs.unstack().reset_index() # unstack(level)이 의미하는 것은 multi Index의 몇번째 index를 칼럼 방향으로 stacking 할것인가를 의미한다
    dst.columns = ['stock_id', 'time_id', f'{feature_col}_cluster{n}{postfix}_{agg.__name__}']
    return dst


In [186]:
temp['seconds_in_bucket']

0           0
1           1
2           5
3           6
4           7
         ... 
917548    568
917549    569
917550    571
917551    572
917552    582
Name: seconds_in_bucket, Length: 917553, dtype: int16

In [191]:
temp.columns[-1]

'C'

In [194]:
temp2 = pd.DataFrame()
temp2[temp.columns[-1]]= temp[temp.columns[-1]].astype(np.float32)

In [195]:
temp2

,C
0,0.720294
1,-0.817565
2,0.309547
3,-1.049585


In [196]:
temp

,A,B,C
0,1,1,0.720294
1,1,2,-0.817565
2,2,3,0.309547
3,2,4,-1.049585


In [21]:
import gc #순환참조를 탐지하고 해결하기 위해 사용하는 모듈
from scipy.stats import rankdata


def rank_data(a, axis=None):
    return a[0] - np.min(a, axis=axis)
# 왜만든거지?

gc.collect()

df2 = df.copy()
print(df2.shape)

df2['real_price'] = 0.01 / df2['tick_size']
del df2['tick_size']

# relative rank
df2['trade.order_count.mean'] = df2.groupby(
    'time_id')['trade.order_count.mean'].rank()
df2['book.total_volume.sum'] = df2.groupby(
    'time_id')['book.total_volume.sum'].rank()
df2['book.total_volume.mean'] = df2.groupby(
    'time_id')['book.total_volume.mean'].rank()
df2['book.total_volume.std'] = df2.groupby(
    'time_id')['book.total_volume.std'].rank()

# time-id 별 trade.order_count와 book의 total_volumne에 대한 sum, mean, std값의 순위를 매긴 컬럼 추가

df2['trade.tau'] = df2.groupby('time_id')['trade.tau'].rank()

#df2['trade_300.tau'] = df2.groupby('time_id')['trade_300.tau'].rank()
#df2['book.total_volume.std'] = df2.groupby('time_id')['book.total_volume.std'].rank()
#df2['trade.size.sum'] = df2.groupby('time_id')['trade.size.sum'].rank()

for dt in [150, 300, 450]:
    df2[f'book_{dt}.total_volume.sum'] = df2.groupby(
        'time_id')[f'book_{dt}.total_volume.sum'].rank()
    df2[f'book_{dt}.total_volume.mean'] = df2.groupby(
        'time_id')[f'book_{dt}.total_volume.mean'].rank()
    df2[f'book_{dt}.total_volume.std'] = df2.groupby(
        'time_id')[f'book_{dt}.total_volume.std'].rank()
    df2[f'trade_{dt}.order_count.mean'] = df2.groupby(
        'time_id')[f'trade_{dt}.order_count.mean'].rank()

# df2.groupby('time_id')['book.volume_imbalance.sum'].rank()
#df2['tick_size'] = df2.groupby('time_id')['tick_size'].rank()

## neighbor stock id 에 대한 feature
feature_cols_stock = {
    'book.log_return1.realized_volatility': [np.mean, np.min, np.max, np.std],

    'trade.seconds_in_bucket.count': [np.mean],
    'trade.tau': [np.mean],
    'trade_150.tau': [np.mean],
    'book.tau': [np.mean],
    'trade.size.sum': [np.mean],
    'book.seconds_in_bucket.count': [np.mean],

    # 'trade.order_count.mean': [np.mean],
    # 'avg_time_vol': [np.mean],
    # 'trade_150.tau': [np.mean],
    # 'trade_450.tau': [np.mean],
    # 'book.total_volume.sum': [np.mean],
    # 'book.volume_imbalance.mean': [np.mean],
}


## neighbor time id 에 대한 feature
feature_cols = {
    'book.log_return1.realized_volatility': [np.mean, np.min, np.max, np.std],
    # 'book_150.log_return1.realized_volatility': [np.mean, np.min],
    # 'book_300.log_return1.realized_volatility': [np.mean, np.min],
    # 'book_450.log_return1.realized_volatility': [np.mean, np.min],

    'real_price': [np.max, np.mean, np.min],

    'trade.seconds_in_bucket.count': [np.mean],
    'trade.tau': [np.mean],
    'trade.size.sum': [np.mean],
    'book.seconds_in_bucket.count': [np.mean],

    # 비슷한 volatility trend를 갖는 20개의 주식종목의 last 300sec에 대한 average tau (average at close time)
    'trade_150.tau_cluster20_sv_mean': [np.mean],
    'trade.size.sum_cluster20_sv_mean': [np.mean],
    # 'book.log_return1.realized_volatility.sq': [np.sum],
    # 'book.estimated_volatility': [np.mean],
    # 'trade.order_count.mean': [np.mean],
    # 'book.total_volume.sum': [np.mean],
    # 'book.volume_imbalance.mean': [np.mean],
    # 'avg_time_vol': [np.mean],
    # 'book.ask_spread.mean': [np.mean],
    # 'book.bid_spread.mean': [np.mean],
}

time_id_neigbor_sizes = [3, 5, 10, 20, 40]
time_id_neigbor_sizes_vol = [2, 3, 5, 10, 20, 40]
stock_id_neighbor_sizes = [10, 20, 40]

ndf = None

cols = []


# ndf가 아무것도 없으면 dst 그대로 반환
# ndf가 있으면 dst의 마지막 열을 ndf에 추가해서 ndf 반환
def _add_ndf(ndf, dst):
    if ndf is None:
        return dst
    else:
        ndf[dst.columns[-1]] = dst[dst.columns[-1]].astype(np.float32)
        return ndf


# neighbor stock_id
for feature_col in feature_cols_stock.keys():
    # feature_pivot, neighbors_stock = make_neighbors_stock(
    #     df2, k_neighbors_stock.neighbors, feature_col, n=N_NEIGHBORS_MAX)
    feature_pivot, neighbors_stock_vol = make_neighbors_stock(
        df2, k_neighbors_stock_vol.neighbors, feature_col, n=N_NEIGHBORS_MAX)
    _, neighbors_stock_size = make_neighbors_stock(
        df2, k_neighbors_stock_size.neighbors, feature_col, n=N_NEIGHBORS_MAX)

    columns = feature_pivot.columns
    index = feature_pivot.index

    for agg in feature_cols_stock[feature_col]:
        for n in stock_id_neighbor_sizes:
            exclude_self = True
            exact = False
            # dst = make_nn_feature(df2, neighbors_stock, columns, index, n=n, agg=agg, postfix='_s',
            #                       exclude_self=exclude_self, exact=exact)
            # ndf = _add_ndf(ndf, dst)
            dst = make_nn_feature(df2, neighbors_stock_vol, columns, index, n=n, agg=agg, postfix='_sv',
                                  exclude_self=exclude_self, exact=exact)
            ndf = _add_ndf(ndf, dst)
            # dst = make_nn_feature(df2, neighbors_stock_size, columns, index, n=n, agg=agg, postfix='_ssize',
            #                     exclude_self=exclude_self)
            #ndf = _add_ndf(ndf, dst)
    del feature_pivot, neighbors_stock_vol

df2 = pd.merge(df2, ndf, on=['time_id', 'stock_id'], how='left')
ndf = None

# neighbor time_id
# ###############################################################################
# for feature_col in feature_cols.keys():
#     feature_pivot, neighbors = make_neighbors(
#         df2, k_neighbors_p2.neighbors, feature_col, n=N_NEIGHBORS_MAX)
#     _, neighbors_p1 = make_neighbors(
#         df2, k_neighbors_p1.neighbors, feature_col, n=N_NEIGHBORS_MAX)
#     _, neighbors_vol = make_neighbors(
#         df2, k_neighbors_vol.neighbors, feature_col, n=N_NEIGHBORS_MAX)
#     _, neighbors_size = make_neighbors(
#         df2, k_neighbors_size.neighbors, feature_col, n=N_NEIGHBORS_MAX)
#     _, neighbors_size_p2 = make_neighbors(
#         df2, k_neighbors_size_p2.neighbors, feature_col, n=N_NEIGHBORS_MAX)

#     columns = feature_pivot.columns
#     index = feature_pivot.index

#     if 'volatility' in feature_col:
#         time_id_ns = time_id_neigbor_sizes_vol
#     else:
#         time_id_ns = time_id_neigbor_sizes

#     for agg in feature_cols[feature_col]:
#         for n in time_id_ns:
#             exclude_self = True  # n >= 10
#             exclude_self2 = False
#             exact = False

#             # if n <= 40:
#             dst = make_nn_feature(df2, neighbors, columns, index, n=n, agg=agg, postfix='_p2',
#                                   exclude_self=exclude_self, exact=exact)
#             ndf = _add_ndf(ndf, dst)
#             dst = make_nn_feature(df2, neighbors_p1, columns, index, n=n, agg=agg, postfix='_p1',
#                                   exclude_self=exclude_self2, exact=exact)
#             ndf = _add_ndf(ndf, dst)

#             dst = make_nn_feature(df2, neighbors_vol, columns, index, n=n, agg=agg, postfix='_v',
#                                   exclude_self=exclude_self2, exact=exact)
#             ndf = _add_ndf(ndf, dst)
#             dst = make_nn_feature(df2, neighbors_size, columns, index, n=n, agg=agg, postfix='_size',
#                                   exclude_self=exclude_self2, exact=exact)
#             ndf = _add_ndf(ndf, dst)
#             dst = make_nn_feature(df2, neighbors_size_p2, columns, index, n=n, agg=agg, postfix='_size_p2',
#                                   exclude_self=exclude_self2, exact=exact)
#             ndf = _add_ndf(ndf, dst)
#             cols.append(dst.columns[-1])

#     del feature_pivot, neighbors, neighbors_p1, neighbors_vol, neighbors_size, neighbors_size_p2

# df2 = pd.merge(df2, ndf, on=['time_id', 'stock_id'], how='left')

# # 주가는 그 자체를 feature로 사용하는게 아니라 time_id 의 nearest를 측정하기 위해 사용됨.

# for sz in time_id_neigbor_sizes:
#     df2[f'real_price_rankmin_{sz}'] = df2['real_price'] / \
#         df2[f"real_price_cluster{sz}_p2_amin"]
#     df2[f'real_price_rankmax_{sz}'] = df2['real_price'] / \
#         df2[f"real_price_cluster{sz}_p2_amax"]
#     df2[f'real_price_rankmean_{sz}'] = df2['real_price'] / \
#         df2[f"real_price_cluster{sz}_p2_mean"]

# for sz in time_id_neigbor_sizes_vol:
#     df2[f'vol_rankmin_{sz}'] = df2['book.log_return1.realized_volatility'] / \
#         df2[f"book.log_return1.realized_volatility_cluster{sz}_p2_amin"]
#     df2[f'vol_rankmax_{sz}'] = df2['book.log_return1.realized_volatility'] / \
#         df2[f"book.log_return1.realized_volatility_cluster{sz}_p2_amax"]

# price_cols = [c for c in df2.columns if 'real_price' in c and 'rank' not in c]
# for c in price_cols:
#     del df2[c]

# #df2['book.log_return1.realized_volatility_rank'] = df2.groupby('time_id')['book.log_return1.realized_volatility'].rank()
# #df2['book.log_return1.realized_volatility_cluster3_size_mean_rank'] = df2.groupby('time_id')['book.log_return1.realized_volatility_cluster3_size_mean'].rank()

# for sz in time_id_neigbor_sizes_vol:
#     tgt = f'book.log_return1.realized_volatility_cluster{sz}_p1_mean'
#     df2[f'{tgt}_rank'] = df2.groupby('time_id')[tgt].rank()

###############################################################################

# skew correction for NN
cols_to_log = [
    'trade.size.sum',
    'trade_150.size.sum',
    'trade_300.size.sum',
    'trade_450.size.sum',
    'volume_imbalance'
]
for c in df2.columns:   
    for check in cols_to_log:
        if check in c:
            df2[c] = np.log(df2[c]+1)
            break

print(df2.shape)
df2.reset_index(drop=True)

del ndf


(428935, 219)
(428935, 249)


In [35]:
df.shape

(428935, 219)

In [32]:
temp_unq = set(df2.columns) - set(df.columns)

In [34]:
len(temp_unq)

31

In [36]:
temp_unq

{'book.log_return1.realized_volatility_cluster10_sv_amax',
 'book.log_return1.realized_volatility_cluster10_sv_amin',
 'book.log_return1.realized_volatility_cluster10_sv_mean',
 'book.log_return1.realized_volatility_cluster10_sv_std',
 'book.log_return1.realized_volatility_cluster20_sv_amax',
 'book.log_return1.realized_volatility_cluster20_sv_amin',
 'book.log_return1.realized_volatility_cluster20_sv_mean',
 'book.log_return1.realized_volatility_cluster20_sv_std',
 'book.log_return1.realized_volatility_cluster40_sv_amax',
 'book.log_return1.realized_volatility_cluster40_sv_amin',
 'book.log_return1.realized_volatility_cluster40_sv_mean',
 'book.log_return1.realized_volatility_cluster40_sv_std',
 'book.seconds_in_bucket.count_cluster10_sv_mean',
 'book.seconds_in_bucket.count_cluster20_sv_mean',
 'book.seconds_in_bucket.count_cluster40_sv_mean',
 'book.tau_cluster10_sv_mean',
 'book.tau_cluster20_sv_mean',
 'book.tau_cluster40_sv_mean',
 'real_price',
 'trade.seconds_in_bucket.count_cl

In [37]:
# 같은 symbol에 대해 비슷한 trading volume을 가진 RV의 평균

df2.sort_values(by=['stock_id', 'book.total_volume.sum'], inplace=True)
df2.reset_index(drop=True, inplace=True)
df2['realized_volatility_roll3_by_book.total_volume.mean'] = df2.groupby('stock_id')['book.log_return1.realized_volatility'].rolling(3, center=True, min_periods=1).mean().reset_index().sort_values(by=['level_1'])['book.log_return1.realized_volatility'].values
df2['realized_volatility_roll10_by_book.total_volume.mean'] = df2.groupby('stock_id')['book.log_return1.realized_volatility'].rolling(10, center=True, min_periods=1).mean().reset_index().sort_values(by=['level_1'])['book.log_return1.realized_volatility'].values

In [40]:
df2

,stock_id,time_id,target,book.seconds_in_bucket.count,book.wap1.sum,book.wap1.mean,book.wap1.std,book.wap2.sum,book.wap2.mean,book.wap2.std,...,book.tau_cluster20_sv_mean,book.tau_cluster40_sv_mean,trade.size.sum_cluster10_sv_mean,trade.size.sum_cluster20_sv_mean,trade.size.sum_cluster40_sv_mean,book.seconds_in_bucket.count_cluster10_sv_mean,book.seconds_in_bucket.count_cluster20_sv_mean,book.seconds_in_bucket.count_cluster40_sv_mean,realized_volatility_roll3_by_book.total_volume.mean,realized_volatility_roll10_by_book.total_volume.mean
0,0,1176,0.005746,144.0,143.815068,0.998716,0.001774,143.849316,0.998954,0.001861,...,0.047374,0.048299,9.806181,10.223156,10.523262,440.666656,465.315796,461.000000,0.004140,0.003038
1,0,8664,0.002469,147.0,146.899894,0.999319,0.000366,146.901871,0.999332,0.000398,...,0.056475,0.055844,9.127394,9.754560,10.304124,316.666656,340.210541,347.589752,0.003729,0.003409
2,0,12758,0.002541,142.0,141.728688,0.998089,0.000900,141.709407,0.997954,0.000960,...,0.050528,0.052232,9.511703,9.618899,9.832170,383.555542,419.157898,402.846161,0.002643,0.002964
3,0,19033,0.002515,94.0,93.842941,0.998329,0.000771,93.848332,0.998387,0.000798,...,0.057823,0.058893,8.882715,9.349086,9.438289,282.444458,328.315796,325.128204,0.002304,0.002818
4,0,20499,0.003066,170.0,169.654033,0.997965,0.000716,169.650958,0.997947,0.000761,...,0.054286,0.054694,9.711109,9.795882,9.855143,350.555542,370.684204,373.282043,0.003089,0.002721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428930,126,16402,0.004163,307.0,307.085638,1.000279,0.002120,307.023339,1.000076,0.002213,...,0.052267,0.056501,9.809000,10.152408,10.100743,419.555542,408.842102,358.410248,0.010648,0.012807
428931,126,4927,0.005008,224.0,224.815252,1.003640,0.000781,224.783674,1.003499,0.000937,...,0.052116,0.054666,10.812129,10.727410,10.774607,397.111115,398.842102,371.538452,0.013510,0.011674
428932,126,15155,0.018900,348.0,337.124379,0.968748,0.006521,336.886673,0.968065,0.006172,...,0.047758,0.050098,11.327068,11.372158,11.323144,476.222229,456.526306,428.641022,0.014948,0.011329
428933,126,13316,0.010262,279.0,281.655248,1.009517,0.002213,281.599159,1.009316,0.002231,...,0.051509,0.056141,11.187591,10.930043,10.774504,405.444458,414.421051,367.743591,0.014979,0.011402


### Stock-id Embedding

자체 pass

In [98]:
# from sklearn.decomposition import LatentDirichletAllocation

# lda_n = 3
# lda = LatentDirichletAllocation(n_components=lda_n, random_state=0)
# stock_id_emb = pd.DataFrame(lda.fit_transform(pivot.transpose()), index=df_pv.pivot('time_id', 'stock_id', 'vol').columns)

# for i in range(lda_n):
#     df2[f'stock_id_emb{i}'] = df2['stock_id'].map(stock_id_emb[i])

In [99]:
# df_train = df2[~df2.target.isnull()].copy()
# df_test = df2[df2.target.isnull()].copy()
# del df2
# gc.collect()b

4235

## Make CV Split

In [100]:
import glob
import time
from contextlib import contextmanager

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

from joblib import Parallel, delayed
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors


%matplotlib inline    

def calc_price2(df):
    tick = sorted(np.diff(sorted(np.unique(df.values.flatten()))))[0]
    return 0.01 / tick


def calc_prices(r):
    df = pd.read_parquet(r.book_path, columns=['time_id', 'ask_price1', 'ask_price2', 'bid_price1', 'bid_price2'])
    df = df.set_index('time_id')
    df = df.groupby(level='time_id').apply(calc_price2).to_frame('price').reset_index()
    df['stock_id'] = r.stock_id
    return df


def sort_manifold(df, clf):
    df_ = df.set_index('time_id')
    df_ = pd.DataFrame(minmax_scale(df_.fillna(df_.mean())))

    X_compoents = clf.fit_transform(df_)

    dft = df.reindex(np.argsort(X_compoents[:,0])).reset_index(drop=True)
    # AMZN
    plt.plot(dft['stock_id=61'])
    plt.plot(dft['stock_id=37'])
    plt.plot(dft['stock_id=113'])
    return np.argsort(X_compoents[:, 0]), X_compoents


def reconstruct_time_id_order():
    with timer('load files'):
        df_files = pd.DataFrame(
            {'book_path': glob.glob('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/**/*.parquet')}) \
            .eval('stock_id = book_path.str.extract("stock_id=(\d+)").astype("int")', engine='python')
        df_target = pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/train.csv')
        df_target = df_target.groupby('time_id').target.mean()

    with timer('calc prices'):
        df_prices = pd.concat(Parallel(n_jobs=4, verbose=51)(delayed(calc_prices)(r) for _, r in df_files.iterrows()))
        df_prices = df_prices.pivot('time_id', 'stock_id', 'price')
        df_prices.columns = [f'stock_id={i}' for i in df_prices.columns]
        df_prices = df_prices.reset_index(drop=False)

    with timer('t-SNE(400) -> 50'):
        clf = TSNE(n_components=1, perplexity=400, random_state=0, n_iter=2000)
        order, X_compoents = sort_manifold(df_prices, clf)
        clf = TSNE(n_components=1, perplexity=50, random_state=0, init=X_compoents, n_iter=2000, method='exact')
        order, X_compoents = sort_manifold(df_prices, clf)

        df_ordered = df_prices.reindex(order).reset_index(drop=True)
        if df_ordered['stock_id=61'].iloc[0] > df_ordered['stock_id=61'].iloc[-1]:
            df_ordered = df_ordered.reindex(df_ordered.index[::-1]).reset_index(drop=True)

    # AMZN
    plt.plot(df_ordered['stock_id=61'])
    
    return df_ordered[['time_id']]

In [101]:
timeid_order = pd.read_csv(os.path.join(DATA_DIR, 'optiver-time-id-ordered', 'time_id_order.csv'))

with timer('make folds'):
    timeid_order['time_id_order'] = np.arange(len(timeid_order))
    df_train['time_id_order'] = df_train['time_id'].map(timeid_order.set_index('time_id')['time_id_order'])
    df_train = df_train.sort_values(['time_id_order', 'stock_id']).reset_index(drop=True)

    folds_border = [3830 - 383*4, 3830 - 383*3, 3830 - 383*2, 3830 - 383*1]
    time_id_orders = df_train['time_id_order']

    folds = []
    for i, border in enumerate(folds_border):
        idx_train = np.where(time_id_orders < border)[0]
        idx_valid = np.where((border <= time_id_orders) & (time_id_orders < border + 383))[0]
        folds.append((idx_train, idx_valid))
        
        print(f"folds{i}: train={len(idx_train)}, valid={len(idx_valid)}")
        
del df_train['time_id_order']

NameError: name 'timeid_order' is not defined